In [83]:
import numpy as np
import pandas as pd
import torch
import wandb
import torchvision.transforms as T
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
from torch.utils.data import WeightedRandomSampler
import torchmetrics

from sklearn.model_selection import train_test_split

from wdd.data_handling.process_data import threshold_data
from wdd.data_handling.pull_data import get_processed_data
from wdd.model.cnn_spp import CNN_SPP_Net,cnn_spp_hypDict
from wdd.data_handling.torch_dataset import WaferDataset
from wdd.data_handling.augment_data import wafer_train_transforms


In [84]:
cnn_channels=(1,3,6,12)
spp_output_sizes=[(1,1),(3,3),(5,5)]
linear_dims=(30,15,9)
model_parameters={'cnn_channels':cnn_channels,'spp_output_sizes':spp_output_sizes,'linear_output_sizes':linear_dims}
net=CNN_SPP_Net(model_parameters)
net.init_weights()

In [4]:
train_df,test_df=get_processed_data()
#split train
train_df,valid_df=train_test_split(train_df, test_size=0.2,random_state=42)

In [5]:
training_set=WaferDataset(train_df,transform=wafer_train_transforms(0.0))
valid_set=WaferDataset(valid_df)

In [6]:
test_set=WaferDataset(test_df)

In [7]:
class_weights=torch.Tensor([1/training_set.len])*torch.Tensor([training_set.y.count(i) for i in range(9)])
assert(np.isclose(class_weights.sum(),1)),'class_weights must sum to be one'

sample_weights=torch.Tensor([1/class_weights[i] for i in training_set.y])

sampler=WeightedRandomSampler(weights=sample_weights,num_samples=len(sample_weights))

In [8]:
valid_class_weights=torch.Tensor([1/valid_set.len])*torch.Tensor([valid_set.y.count(i) for i in range(9)])
assert(np.isclose(valid_class_weights.sum(),1)),'valid_class_weights must sum to be one'

In [9]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=1 , num_workers=0,sampler=sampler)

In [10]:
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=1, shuffle=True, num_workers=0)

In [11]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True, num_workers=0)

In [12]:
from torch.optim import Adam
 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
train_loss_fn = torch.nn.CrossEntropyLoss()
valid_loss_fn = torch.nn.CrossEntropyLoss(weight=valid_class_weights.reciprocal())
optimizer = Adam(net.parameters(), lr=0.001, weight_decay=0.0001)

In [13]:
from wdd.model.model_training import train_model

train_model(
    net,
    training_loader,
    valid_loader,
    train_loss_fn,
    valid_loss_fn,
    optimizer,
    epochs=5
)

EPOCH 1:
LOSS train 3.9106830650756264 valid 1.92219078540802
EPOCH 2:
LOSS train 2.6111099877962496 valid 1.2526944875717163
EPOCH 3:
LOSS train 1.9641822745589794 valid 0.9713939428329468
EPOCH 4:
LOSS train 1.7516204972800673 valid 0.9831929802894592
EPOCH 5:
LOSS train 1.6111272861566168 valid 0.928584098815918


In [14]:
y_trues,y_preds,y_pred_probs = net.predict(test_loader)

/mnt/c/Users/lslat/Code/wafer-defect-detection/src/wdd/model/cnn_spp.py:112: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits=torch.nn.Softmax()(x)


In [15]:
acc=torchmetrics.Accuracy(num_classes=9,average='micro')
bacc=torchmetrics.Accuracy(num_classes=9,average='macro')
by_class_acc=torchmetrics.Accuracy(num_classes=9,average='none')
f1=torchmetrics.F1Score(num_classes=9,average='micro')
bf1=torchmetrics.F1Score(num_classes=9,average='macro')
by_class_f1=torchmetrics.F1Score(num_classes=9,average='none')


In [16]:
acc(y_trues,y_preds)

tensor(0.7506)

In [17]:
by_class_acc(y_trues,y_preds)

tensor([0.9815, 0.1583, 0.2194, 0.5261, 0.8978, 0.0618, 0.4330, 0.6053, 0.7000])

In [18]:
f1(y_trues,y_preds)

tensor(0.7506)

In [19]:
bf1(y_trues,y_preds)

tensor(0.5706)

In [20]:
by_class_f1(y_trues,y_preds)

tensor([0.8508, 0.2558, 0.2460, 0.6307, 0.9161, 0.1033, 0.5957, 0.7132, 0.8235])

In [58]:
tuple(9*2**((i-1)) for i in range(5,0,-1))

(144, 72, 36, 18, 9)

In [48]:
(3-1)//2

1

In [35]:
[(1+2*i,1+2*i) for i in range(3)]

[(1, 1), (3, 3), (5, 5)]

In [53]:
type(valid_loss_fn)

torch.nn.modules.loss.CrossEntropyLoss

In [54]:
type(valid_loader)

torch.utils.data.dataloader.DataLoader